In [4]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from datetime import datetime
import os
import sys
import json
import requests
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import threading

In [5]:
def categorias():
    categorias_l=[]
    enlaces_l=[]
    url='https://www.carrefour.es/supermercado/productos-frescos/cat20002/c'
    pag=urllib.request.urlopen(url)
    doc=BeautifulSoup(pag)
    for i in doc.find_all("div",class_="nav-first-level-categories__slide"):
        if i.get("title")=="Mis productos" or i.get("title")=="Ofertas":
            pass
        else:
            categorias_l.append(i.get("title"))
            enlaces_l.append("https://www.carrefour.es"+i.find("a").get("href"))

            print(i.get("title"),"https://www.carrefour.es"+i.find("a").get("href"))

    return enlaces_l,categorias_l
[enlaces_l,categorias_l]=categorias()

Productos Frescos https://www.carrefour.es/supermercado/productos-frescos/cat20002/c
La Despensa https://www.carrefour.es/supermercado/la-despensa/cat20001/c
Congelados https://www.carrefour.es/supermercado/congelados/cat21449123/c
Bebidas https://www.carrefour.es/supermercado/bebidas/cat20003/c
Limpieza y Hogar https://www.carrefour.es/supermercado/limpieza-y-hogar/cat20005/c
Perfumería e Higiene https://www.carrefour.es/supermercado/perfumeria-e-higiene/cat20004/c
Bebé https://www.carrefour.es/supermercado/bebe/cat20006/c
Mascotas https://www.carrefour.es/supermercado/mascotas/cat20007/c
Parafarmacia https://www.carrefour.es/supermercado/parafarmacia/cat20008/c


In [6]:
def productos(enlaces_l,categorias_l):
    
    for idx,i in enumerate(enlaces_l):
        url=i
        pag=urllib.request.urlopen(url)
        doc_1=BeautifulSoup(pag)
        for jdx,j in enumerate(doc_1.find_all("div",class_="nav-second-level-categories__slide")):
            if jdx==0:
                pass
            else:
                
                print(categorias_l[idx],j.find("a").get("href"))
                enlace_2="https://www.carrefour.es"+j.find("a").get("href")
                

                categoria_final=[]
                nombre=[]
                precio=[]
                precio_unidad=[]
                subcategoria=[]
                vistos=[]
                pagina=0
                check=0
                while pagina>=0:
                    try:
                        if pagina==0:
                            enlace_2=enlace_2
                            driver = webdriver.Chrome()
                            
                            driver.get(enlace_2)
                            time.sleep(2)
                            driver.find_element(By.ID,'onetrust-accept-btn-handler').click()
                            #driver.find_element(By.CLASS_NAME,'icon-cross-thin').click()
                            for scroll in range(15):
                                driver.find_element(By.CSS_SELECTOR,'body').send_keys(Keys.PAGE_DOWN)
                            time.sleep(2)
                            pag=driver.page_source
                            doc_2=BeautifulSoup(pag,'html.parser')
                            driver.close()
                            
                        else:
                            if check==0:
                                enlace_2="https://www.carrefour.es"+j.find("a").get("href")+"?offset="+str(pagina)
                            else:
                                pass
                            driver = webdriver.Chrome()
                            
                            driver.get(enlace_2)
                            time.sleep(2)
                            driver.find_element(By.ID,'onetrust-accept-btn-handler').click()
                            #driver.find_element(By.CLASS_NAME,'icon-cross-thin').click()
                            for scroll in range(15):
                                driver.find_element(By.CSS_SELECTOR,'body').send_keys(Keys.PAGE_DOWN)
                            time.sleep(2)
                            pag=driver.page_source
                            doc_2=BeautifulSoup(pag,'html.parser')
                            driver.close()
                    except:
                        print("error")
                        continue
                    print(enlace_2)
                    
                    print(len(doc_2.find_all("li",class_="product-card-list__item")))
                    
                    if  len(doc_2.find_all("li",class_="product-card-list__item"))==0:
                        check=1
                        print("intetnado otra vez")
                        pass
                    elif len(doc_2.find_all("li",class_="product-card-list__item"))<25:
                        pagina=-1
                        print("saliendo")
                        break
                    
                    else:
                        check=0
                        for k in doc_2.find_all("li",class_="product-card-list__item"):
                        #time.sleep(1)
                            if k.find("span",class_="product-card__price")==None:
                                pass
                            else:
                                #print(k.find("a",class_="product-card__title-link track-click").get_text(),k.find("span",class_="product-card__price").get_text(),k.find("span",class_="product-card__price-per-unit").get_text())
                                #print(k.find("a",class_="product-card__title-link track-click").get_text(),k.find("span",class_="product-card__price"))
                                categoria_final.append(categorias_l[idx])
                                nombre.append(k.find("a",class_="product-card__title-link track-click").get_text())
                                precio.append(k.find("span",class_="product-card__price").get_text())
                                if k.find("span",class_="product-card__price-per-unit")!=None:
                                    precio_unidad.append(k.find("span",class_="product-card__price-per-unit").get_text())
                                else:
                                    precio_unidad.append(0)
                                subcategoria.append(j.find("a").get("href"))
                                #print(k.find("a",class_="product-card__title-link track-click").get_text())
                        if doc_2.find("span",class_="pagination__next icon-right-arrow-thin")==None:
                            pagina=-1
                        else:

                            vistos.append("https://www.carrefour.es"+j.find("a").get("href")+"?offset="+str(pagina))
                            pagina=pagina+24
                                
                new_df=pd.DataFrame(data={
                    "nombre":nombre,
                    "categoria":categoria_final,
                    "subcategoria":subcategoria,
                    "precio":precio,
                    "precio_unidad":precio_unidad,
                    "comercio":"Carrefour",
                    "fecha":str(datetime.today()).split(" ")[0]

                })
                
                if os.path.isfile("carrefour.csv")==True:
                    datos=pd.read_csv("carrefour.csv",sep="|",index_col=False)
                    final_df=pd.concat([datos,new_df],axis=0)
                    final_df.to_csv("carrefour.csv",sep="|",index=False,encoding="utf-8")
                else:
                    new_df.to_csv("carrefour.csv",sep="|",index=False,encoding="utf-8")                 
                          
                        
                            
                    

productos(enlaces_l,categorias_l)



Productos Frescos /supermercado/productos-frescos/carniceria/cat20018/c
error
https://www.carrefour.es/supermercado/productos-frescos/carniceria/cat20018/c
25
error
error
error
https://www.carrefour.es/supermercado/productos-frescos/carniceria/cat20018/c?offset=24
25


In [ ]:
#threading.Thread(target=productos, args=(enlaces_l[:3],categorias_l)).start()
#threading.Thread(target=productos, args=(enlaces_l[3:],categorias_l)).start()
